In [11]:
from bs4 import BeautifulSoup
import requests
import json

# Parsing
## Get list of all the sections

In [22]:
headers = requests.utils.default_headers()
url = "https://www.tesco.com/groceries/en-GB/shop/?include-children=true"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')
result = soup.find_all("script", type="application/ld+json")
list_ = json.loads(result[0].string)
sections_urls_list = []
for next_ in list_:
    if next_["@type"] == "ItemList":
        elements_list = next_["itemListElement"]
        sections_urls_list = list(map(lambda x: x["url"], elements_list))
        break
print(sections_urls_list)



['https://www.tesco.com/groceries/en-GB/shop/fresh-food', 'https://www.tesco.com/groceries/en-GB/shop/bakery', 'https://www.tesco.com/groceries/en-GB/shop/frozen-food', 'https://www.tesco.com/groceries/en-GB/shop/food-cupboard', 'https://www.tesco.com/groceries/en-GB/shop/drinks', 'https://www.tesco.com/groceries/en-GB/shop/baby', 'https://www.tesco.com/groceries/en-GB/shop/health-and-beauty', 'https://www.tesco.com/groceries/en-GB/shop/pets', 'https://www.tesco.com/groceries/en-GB/shop/household', 'https://www.tesco.com/groceries/en-GB/shop/home-and-ents', 'https://www.tesco.com/groceries/en-GB/shop/easter', 'https://www.tesco.com/groceries/en-GB/shop/inspiration-and-events']


## Handling content inside sections

In [75]:
#parse for products urls on the page
def parse_page_content(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    result = soup.find_all("script", type="application/ld+json")
    list_ = json.loads(result[0].string)
    for next_ in list_:
        if next_["@type"] == "ItemList":
            elements_list = next_["itemListElement"]
            sections_urls_list = list(map(lambda x: x["url"], elements_list))
            break
    print(sections_urls_list)

def find_last_page_index(url):
    req = requests.get(url, headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    #try to get the last page index:
    result = soup.find_all("li", class_="pagination-btn-holder")
    #-1 is going back to results, -2 - is the last page
    last_page = result[-2].find_all("span")[0].string
    return int(last_page)

    
#Finds last page and call parsing for each page
def parse_section_content(url):
    last_page = find_last_page_index(url)
    #cause I don't want to scrape everything, most probably we will need to have some delays for prevent bot detection checks
    i = 0
    for next_page in range(1, last_page+1):
        url = url+"?page=" + str(next_page)
        parse_page_content(url)
        i = i + 1
        if i == 3:
            return
        


In [76]:
for next_url in sections_urls_list:
    parse_section_content(next_url+"/all")
    break

['https://www.tesco.com/groceries/en-GB/products/290920510', 'https://www.tesco.com/groceries/en-GB/products/290921181', 'https://www.tesco.com/groceries/en-GB/products/284475671', 'https://www.tesco.com/groceries/en-GB/products/296920881', 'https://www.tesco.com/groceries/en-GB/products/260138854', 'https://www.tesco.com/groceries/en-GB/products/258423755', 'https://www.tesco.com/groceries/en-GB/products/266419328', 'https://www.tesco.com/groceries/en-GB/products/288135169', 'https://www.tesco.com/groceries/en-GB/products/266499399', 'https://www.tesco.com/groceries/en-GB/products/267459342', 'https://www.tesco.com/groceries/en-GB/products/252207537', 'https://www.tesco.com/groceries/en-GB/products/295673143', 'https://www.tesco.com/groceries/en-GB/products/287529333', 'https://www.tesco.com/groceries/en-GB/products/253556398', 'https://www.tesco.com/groceries/en-GB/products/280046682', 'https://www.tesco.com/groceries/en-GB/products/285157326', 'https://www.tesco.com/groceries/en-GB/